In [ ]:
# Run
import yfinance as yf
import pandas as pd
import datetime
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

off_limits = ["NVDA","AAPL","MSFT","GOOG","AMZN","COST","V","UBER","NOW","META","TSLA","CRWD","AMD","SNOW", "LRCX", "PANW","DE","JPM","HD","JNJ","AVGO"]

# Pull Core & Div Holdings
try:
    excel_path = r"F:\\WorkGrp\\Investments\\Equities\\Stock Analysis\\Core Port and Dvd Port.xlsx"
    core_sheet = pd.read_excel(open(excel_path, 'rb'), sheet_name='Core Holdings',header=None, skiprows =[i for i in range(1, 3) ],index_col=4)
    core_sheet = core_sheet.iloc[:, 4:8]
    core_sheet = core_sheet.reset_index().dropna()
    core_sheet = core_sheet.iloc[:, :2]
    core_sheet.columns = ["tickers","shares"]
    core_sheet = core_sheet[pd.to_numeric(core_sheet['shares'], errors='coerce').notnull()]
    core_sheet = core_sheet.drop(core_sheet[core_sheet.shares <= 0].index)
    tickers1 = core_sheet.tickers.to_list()

    dvd_sheet = pd.read_excel(open(excel_path, 'rb'), sheet_name='Dvd Holdings',header=None, skiprows =[i for i in range(1, 3) ],index_col=6)
    dvd_sheet = dvd_sheet.iloc[:, 6:10]
    dvd_sheet = dvd_sheet.reset_index().dropna()
    dvd_sheet = dvd_sheet.iloc[:, :2]
    dvd_sheet.columns = ["tickers","shares"]
    dvd_sheet = dvd_sheet[pd.to_numeric(dvd_sheet['shares'], errors='coerce').notnull()]
    dvd_sheet = dvd_sheet.drop(dvd_sheet[dvd_sheet.shares <= 0].index)
    tickers2 = dvd_sheet.tickers.to_list()

    current_holdings = tickers1 + tickers2
    try:
        current_holdings = [ticker for ticker in current_holdings if ticker not in off_limits]
    except:
        pass
except:
  print("Current Portfolio Holdings are Unavailable")
  current_holdings = []

# Convert Time
def convert_to_datetime(date_obj):
    if isinstance(date_obj, datetime.date) and not isinstance(date_obj, datetime.datetime):
        return datetime.datetime.combine(date_obj, datetime.datetime.min.time())
    return date_obj

# Define widgets
check_widget = widgets.Checkbox(
    value=False,
    description='Filter Current Holdings',
    disabled=False,
    indent=False
)
tickers_widget = widgets.Text(
    value="",
    description='Tickers:',
    style={'description_width': 'initial'}
)
cushion_widget = widgets.FloatSlider(
    value=0.30,
    min=0,
    max=0.5,
    step=0.005,
    description='Cushion (%):',
    style={'description_width': 'initial'}
)
min_return_widget = widgets.FloatSlider(
    value=0.00,
    min=0,
    max=25,
    step=0.10,
    description='Min Premium:',
    style={'description_width': 'initial'}
)
volume_min_widget = widgets.IntSlider(
    value=5,
    min=0,
    max=100,
    step=1,
    description='Volume Min:',
    style={'description_width': 'initial'}
)
min_premium_all_widget = widgets.IntText(
    value=0,
    description='Min Premium (All):',
    style={'description_width': 'initial'}
)
expiration_start_widget = widgets.DatePicker(
    value=datetime.datetime(2024, 7, 1),
    description='Expiration Start:',
    style={'description_width': 'initial'}
)
expiration_end_widget = widgets.DatePicker(
    value=datetime.datetime(2024, 9, 30),
    description='Expiration End:',
    style={'description_width': 'initial'}
)

sort_by_widget = widgets.Dropdown(
    options=['bid', 'cushion (%)',  'implied_volatility', 'max_profit', 'max_premium (all_shares)'],
    value='bid',
    description='Sort By:',
    style={'description_width': 'initial'}
)
sort_order_widget = widgets.Checkbox(
    value=False,
    description='Ascending Order',
    disabled=False,
    indent=False
)

refresh_button = widgets.Button(
    description='Refresh',
    button_style='success',
    tooltip='Click to refresh data',
    icon='refresh'
)

# Display
widget_group1 = widgets.HBox([tickers_widget,check_widget, ])
widget_group2 = widgets.VBox([cushion_widget, min_return_widget, volume_min_widget, min_premium_all_widget, expiration_start_widget, expiration_end_widget])
widget_group3 = widgets.HBox([sort_by_widget, sort_order_widget])
display(widget_group1, widget_group2, widget_group3, refresh_button)

loading_text = widgets.HTML("<em>Loading...</em>")

# Filter call options
def filter_call_options(tickers, expiration_start, expiration_end, cushion, min_return, volume_min, min_premium_all):
    try:
      tickers = [ticker.strip() for ticker in tickers.split(",")]
    except:
      pass
    options_data = []

    if not tickers:
        return pd.DataFrame(columns=['ticker', 'contract_name', 'expiration_date', 'strike', 'current_price', 'cushion (%)', 'bid', 'volume', 'open_interest', 'implied_volatility', 'premium_received', 'max_profit', 'premium/price', 'max_premium (all_shares)'])

    for ticker in tickers:
        try:
          if ticker in core_sheet['tickers'].values:
              shares = core_sheet.loc[core_sheet['tickers'] == ticker, 'shares'].values[0]
          elif ticker in dvd_sheet['tickers'].values:
              shares = dvd_sheet.loc[dvd_sheet['tickers'] == ticker, 'shares'].values[0]
          else:
              shares = 0
        except:
          shares = 0
          pass

        if not ticker:
            return pd.DataFrame(columns=['ticker', 'contract_name', 'expiration_date', 'strike', 'current_price', 'cushion (%)', 'bid', 'volume', 'open_interest', 'implied_volatility', 'premium_received', 'max_profit', 'premium/price', 'max_premium (all_shares)'])
        ticker = ticker.replace(" ","")
        stock = yf.Ticker(ticker)

        try:
            current_price = stock.info.get('currentPrice') #stock.history(period="1d")['Close'].iloc[-1]
        except IndexError:
            #print(f"No price data found for {ticker}. It may be delisted.")
            continue

        expiration_start = convert_to_datetime(expiration_start)
        expiration_end = convert_to_datetime(expiration_end)

        expiration_dates = stock.options
        expiration_dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in expiration_dates]

        expiration_dates = [date.strftime('%Y-%m-%d') for date in expiration_dates if expiration_start <= date <= expiration_end]
        for exp_date in expiration_dates:
            opt_chain = stock.option_chain(exp_date)
            calls = opt_chain.calls
            upper_price = current_price * (1 + cushion)

            # Filter calls
            filtered_calls = calls[(calls['strike'] >= upper_price)].copy()
            if not filtered_calls.empty:
                filtered_calls = filtered_calls[filtered_calls['openInterest'] > 1]
                filtered_calls = filtered_calls[filtered_calls['volume'] >= volume_min]
                if not filtered_calls.empty:
                    filtered_calls['days_to_expiration'] = (datetime.datetime.strptime(exp_date, '%Y-%m-%d') - datetime.datetime.now()).days
                    filtered_calls['potential_return'] = filtered_calls['bid'] / current_price
                    filtered_calls['max_premium (all_shares)'] = round(filtered_calls['bid'] * shares, 2)
                    if not filtered_calls.empty:
                        filtered_calls['type'] = 'call'
                        filtered_calls['current_price'] = round(current_price, 2)
                        filtered_calls['ticker'] = ticker
                        filtered_calls['expiration_date'] = exp_date
                        filtered_calls['cushion (%)'] = round(((filtered_calls['strike'] / current_price) - 1) * 100, 3)
                        filtered_calls['premium_received'] = filtered_calls['bid'] * 100
                        filtered_calls['implied_volatility'] = round(filtered_calls['impliedVolatility'] * 100, 3)
                        filtered_calls['max_profit'] = (filtered_calls['premium_received'] + ((filtered_calls['strike'] - current_price) * 100))
                        filtered_calls['premium/price'] = round(filtered_calls['premium_received'] / current_price, 2)
                        filtered_calls = filtered_calls[filtered_calls['bid'] >= float(min_return)]
                        filtered_calls = filtered_calls[filtered_calls['max_premium (all_shares)'] >= min_premium_all]
                        filtered_calls.rename(columns={'contractSymbol': 'contract_name', 'openInterest': 'open_interest'}, inplace=True)
                        options_data.append(filtered_calls)

    if options_data:
        options_df = pd.concat(options_data, ignore_index=True)
        #options_df = options_df.sort_values(by='bid', ascending=False)
        options_df = options_df[['ticker', 'contract_name', 'expiration_date', 'strike', 'current_price', 'cushion (%)', 'bid', 'volume', 'open_interest', 'implied_volatility', 'premium_received', 'max_profit', 'premium/price','max_premium (all_shares)']]
        return options_df.reset_index(drop=True)
    else:
        return pd.DataFrame(columns=['ticker', 'contract_name', 'expiration_date', 'strike', 'current_price', 'cushion (%)', 'bid', 'volume', 'open_interest', 'implied_volatility', 'premium_received', 'max_profit', 'premium/price','max_premium (all_shares)'])

def round_columns(df):
    try:
        return df.round(2)
    except Exception as e:
        print(f"Error rounding columns: {e}")
        return df

def format_and_display(df):
    return df.style.format(
        {
            'strike': '{:.2f}',
            'current_price': '{:.2f}',
            'cushion (%)': '{:.2f}',
            'bid': '{:.2f}',
            'volume': '{:.2f}',
            'open_interest': '{:.2f}',
            'implied_volatility': '{:.2f}',
            'premium_received': '{:,.0f}',
            'max_profit': '{:,.0f}',
            'premium/price': '{:.2f}',
            'max_premium (all_shares)': '{:,.0f}'
        }
    ).set_table_attributes('style="font-size: 12px"').set_properties(**{'border': '1px solid black'}).bar(subset=['cushion (%)'], color='lightblue').bar(subset=['max_profit'], color='lightgreen').bar(subset=['implied_volatility'], color='lightcoral')

# Update function
def update_data(button):
    filter_widget.clear_output()
    filter_group_widget.clear_output()

    with filter_widget:
        display(loading_text)

    holdings_check = check_widget.value
    tickers = tickers_widget.value
    cushion = cushion_widget.value
    min_return = min_return_widget.value
    volume_min = volume_min_widget.value
    min_premium_all = min_premium_all_widget.value
    expiration_start = expiration_start_widget.value
    expiration_end = expiration_end_widget.value

    if holdings_check == True:
      tickers = current_holdings

    filtered_call_options = filter_call_options(tickers, expiration_start, expiration_end, cushion, min_return, volume_min, min_premium_all)
    filtered_call_options = round_columns(filtered_call_options)

    sort_by = sort_by_widget.value
    ascending_order = sort_order_widget.value
    filtered_call_options = filtered_call_options.sort_values(by=sort_by, ascending=ascending_order)

    filter_widget.clear_output()
    filter_group_widget.clear_output()

    with filter_widget:
        display(widgets.HTML(f"<h3>ALL TICKERS</h3>"))
        display(format_and_display(filtered_call_options))

    grouped_options = filtered_call_options.groupby('ticker')
    with filter_group_widget:
        for ticker, group in grouped_options:
            display(widgets.HTML(f"<h3>{ticker}</h3>"))
            display(format_and_display(group))

refresh_button.on_click(update_data)

# Initial call
filter_widget = widgets.Output()
display(filter_widget)
filter_group_widget = widgets.Output()
display(filter_group_widget)
update_data(None)

Button(button_style='success', description='Refresh', icon='refresh', style=ButtonStyle(), tooltip='Click to r…

Output()

Output()